# 2. Redes Neurais Profundas

Uma rede neural profunda do tipo Feed Forward Multilayer Perceptron para classificação de tweets na polaridade positiva, negativa e neutra.


In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
from torch.utils.data import Dataset, DataLoader
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import classification_report

In [2]:
class CustomDataset(Dataset):
    def __init__(self, data, labels):
        self.data = data
        self.labels = labels

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx], self.labels[idx]

In [3]:
class MLPClassifier(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(MLPClassifier, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, hidden_size)
        self.fc3 = nn.Linear(hidden_size, output_size)
        self.relu = nn.ReLU()
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.fc2(x)
        x = self.fc3(x)
        return self.softmax(x)

Carregar dados do arquivo CSV usando pandas

In [4]:
df_train = pd.read_csv('https://raw.githubusercontent.com/giacicunb/enap_pln2024/main/corpora/twitter-2016train-A.txt',sep='\t',encoding="UTF-8")
df_test = pd.read_csv('https://raw.githubusercontent.com/giacicunb/enap_pln2024/main/corpora/twitter-2016test-A.txt',sep='\t',encoding="UTF-8")

Número de classes (são 3 tipos de polaridade de tweets)

In [5]:
num_classes = len(df_train['polarity'].unique())

Obtém os tweets e as labels separadamente para os dados de treinamento e de teste

In [6]:
df_train['polarity'] = pd.Categorical(df_train['polarity'])
df_train['polarity'] = df_train['polarity'].cat.codes

df_test['polarity'] = pd.Categorical(df_test['polarity'])
df_test['polarity'] = df_test['polarity'].cat.codes

train_tweets = df_train['text'].tolist()
train_labels = df_train['polarity'].tolist()

test_tweets = df_test['text'].tolist()
test_labels = df_test['polarity'].tolist()

Calcula os vetores TF-IDF para os dados de treinamento e de teste

In [7]:
tfidf_vectorizer = TfidfVectorizer()
train_tfidf = tfidf_vectorizer.fit_transform(train_tweets).toarray()

test_tfidf = tfidf_vectorizer.transform(test_tweets).toarray()

Converter os vetores TF-IDF para tensores

In [8]:
train_tensor = torch.tensor(train_tfidf, dtype=torch.float32)
test_tensor = torch.tensor(test_tfidf, dtype=torch.float32)

torch_train_labels = torch.tensor(train_labels, dtype=torch.long)
torch_test_labels = torch.tensor(test_labels, dtype=torch.long)

Criar instâncias do DataLoader

In [9]:
train_dataset = CustomDataset(train_tensor, torch_train_labels)
test_dataset = CustomDataset(test_tensor, torch_test_labels)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

Definindo alguns hiperparâmetros:

*   Tamanho do vetor TF-IDF de entrada
*   Quantidade de neurônios na camada oculta
*   Dimensão da camada de saída



In [10]:
input_size = train_tensor.shape[1]
hidden_size = 128
output_size = num_classes

Instancia o objeto referente a rede neural profunda

In [11]:
dnn_model = MLPClassifier(input_size, hidden_size, output_size)

Define a função loss e o otimizador de Adam para otimização dos parâmetros

In [12]:
loss_function = nn.CrossEntropyLoss()
optimizer = optim.Adam(dnn_model.parameters(), lr=0.001)

Loop de treinamento do modelo

In [13]:
num_epochs = 30
for epoch in range(num_epochs):

    dnn_model.train()

    total_loss = 0
    for text, labels in train_loader:

        optimizer.zero_grad()
        outputs = dnn_model(text)

        loss = loss_function(outputs, labels)

        loss.backward()
        optimizer.step()

        total_loss += loss.item()
    print(f'Epoch {epoch+1}, Loss: {total_loss/len(train_loader)}')

Epoch 1, Loss: 0.9853490377083803
Epoch 2, Loss: 0.6844639739929101
Epoch 3, Loss: 0.24764725470390075
Epoch 4, Loss: 0.04763648906149543
Epoch 5, Loss: 0.013493162541029355
Epoch 6, Loss: 0.005659616397072871
Epoch 7, Loss: 0.0028557573864534977
Epoch 8, Loss: 0.001659266006661197
Epoch 9, Loss: 0.0011676368046140012
Epoch 10, Loss: 0.0008304033919837541
Epoch 11, Loss: 0.0006240571448683906
Epoch 12, Loss: 0.0004702054245101933
Epoch 13, Loss: 0.0003707977030456114
Epoch 14, Loss: 0.0002939258876535743
Epoch 15, Loss: 0.00023848560410372628
Epoch 16, Loss: 0.00019609629005292215
Epoch 17, Loss: 0.0001610152416665537
Epoch 18, Loss: 0.0001359080528438939
Epoch 19, Loss: 0.00011437888175490886
Epoch 20, Loss: 9.78592943575234e-05
Epoch 21, Loss: 8.504011695829552e-05
Epoch 22, Loss: 7.082453492283747e-05
Epoch 23, Loss: 6.141968662212513e-05
Epoch 24, Loss: 5.3908261156137974e-05
Epoch 25, Loss: 4.670423230051641e-05
Epoch 26, Loss: 4.278225644436763e-05
Epoch 27, Loss: 3.6546576421432

In [16]:
y_pred = []
y_test = []
for text, labels in test_loader:
    y_prob = dnn_model(text)
    _, predicted = torch.max(y_prob, 1)
    y_pred.extend(predicted.tolist())
    y_test.extend(labels.tolist())

In [17]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.30      0.25      0.27      3231
           1       0.56      0.37      0.45     10342
           2       0.44      0.68      0.53      7059

    accuracy                           0.46     20632
   macro avg       0.43      0.43      0.42     20632
weighted avg       0.48      0.46      0.45     20632

